In [1]:
import pandas as pd

In [9]:
df = pd.read_excel('Exlibri.xlsx', names=['signatur','jahrn','jahrz','artv','artn','artidn','besitv','besitn','beidn','titel','ort','umfang','blatt','bild','platte','technik','technikidn','bemerk'], dtype='string')
df.head()

,signatur,jahrn,jahrz,artv,artn,artidn,besitv,besitn,beidn,titel,ort,umfang,blatt,bild,platte,technik,technikidn,bemerk
0,1988/Bl. 750,1988,vor 1988,<NA>,<NA>,<NA>,C. G.,Schönkopf,<NA>,"Exlibris-Radierung mit Flasche auf Podest, umr...",<NA>,,290 x 198,30 x 30,45 x 45,Radierung,<NA>,<NA>
1,1981/Bl. 117,1981,vor 1981,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Exlibris mit im Stil von Vorsatzpapier in Wabe...,<NA>,<NA>,150 x 90,<NA>,<NA>,<NA>,<NA>,<NA>
2,1966/Bl. 147,1601,1601-1700,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Exlibris mit viergeteiltes Wappenschild - Löwe...,<NA>,<NA>,205 x 180,<NA>,185 x 170,Kupferstich,<NA>,<NA>
3,1966/Bl. 149,1701,1701 - 1800,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Exlibris mit Sinnspruch: In aeternum non commo...,<NA>,<NA>,111 x 132,92 x 111,<NA>,Kupferstich,<NA>,<NA>
4,1966/Bl. 158,1701,1701 - 1800,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Exlibris mit Devise: Adjuvante Servatore : Mut...,<NA>,<NA>,93 x 63,<NA>,<NA>,Kupferstich,<NA>,<NA>


In [12]:
def pica_schreiben(row):

    pica = f"""0500 Pa
0501 $bsti
0502 $bn
0503 $bnb
0600 yy
"""
    if pd.notna(row.jahrz):
        pica += f"1100 {row.jahrn}$n{row.jahrz}\n"
    else:
        pica += f"1100 {row.jahrn}\n"
    
    pica += f"""1130 TB-papier
1132 a1-analog;f1-bild;f2-blatt;e2-uv
"""

    if pd.notna(row.artidn):
        pica += f"3010 {row.artidn}$BKünstler$4art\n"
    
    if pd.isna(row.artidn) and pd.notna(row.artn):
        pica += f"3010 {row.artn}, {row.artv}$BKünstler$4art\n"

    if pd.notna(row.beidn):
        pica += f"3010 {row.beidn}$BFrüherer Eigentümer$4fmo\n"

    if pd.isna(row.beidn) and pd.notna(row.besitn):
        pica += f"3010 {row.besitn}, {row.besitv}$BFrüherer Eigentümer$4fmo\n"
    
    pica += f"4000 {row.titel}"

    if pd.notna(row.artn):
        pica += f" \ {row.artn}, {row.artv}"

    if pd.notna(row.besitn):
        pica += f" ; für {row.besitn}, {row.besitv}"
    pica += f"\n"

    return pica


    




In [ ]:
for index, row in df.sample(18).iterrows():
    print(pica_schreiben(row))